# 环境准备

## MiniConda

Guideline: https://docs.anaconda.com/miniconda/

```
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm ~/miniconda3/miniconda.sh

source ~/miniconda3/bin/activate

conda init --all
```

## VSCode & Jupyter

```
pip install ipykernel -U --user --force-reinstall
```


In [ ]:
print('Hello Python')

hi


## Check Version

```
# 查询镜像中的cuda版本
ldconfig -p | grep cuda

# 查询镜像中的cudnn版本
ldconfig -p | grep cudnn

conda list | grep cudatoolkit

conda list | grep cudnn

```

KB: 用 conda 安装

```
conda search cudatoolkit

conda install cudatoolkit==xx.xx
conda install cudnn==xx.xx

```

KB: 下载安装

https://developer.nvidia.com/cuda-toolkit-archive

https://developer.nvidia.com/cudnn




## Pytorch & Cuda

Ref: https://pytorch.org/get-started/previous-versions/

```
conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.6 -c pytorch -c nvidia
```